In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
#Example ADS-B data gathered from flightradar24
flight = pd.read_csv('data/24299085.csv')

print(list(flight))

In [ ]:
#We can use simple accessor methods to find out useful data
flight['Altitude'].min(),flight['Altitude'].max()

In [ ]:
#We can do more complex operations like finding the difference between each successive pair of value
flight['diff'] = flight['Altitude'].diff()

In [ ]:
flight['diff'].min(),flight['diff'].max()

In [ ]:
#We can also create functions to apply to our data
def energy(speed,altitude,mass,gravity=9.81):
    ke=0.5*mass*speed**2
    pe=mass*gravity*altitude
    return ke+pe

In [ ]:
#Using 5000kg as reasonably wrong answer for Jetstream 31
print(energy(flight['Speed'],flight['Altitude'],5000))

In [ ]:
#We can also apply it in a vectorised fashion on our pandas dataframe which is more efficent if your data is large
mass=5000
gravity=9.81
flight['ke'] = flight.apply(lambda r: 0.5*mass*r['Speed']**2,axis=1)
flight['pe'] = flight.apply(lambda r: gravity*mass*r['Altitude'],axis=1)

In [ ]:
flight.head()

In [ ]:
flight['Datetime'] = pd.to_datetime(flight['UTC'],utc=True)

In [ ]:
fig, axes = plt.subplots(3,1,sharex='all')
axes[0].plot(flight['Datetime'],flight['Altitude'])
axes[0].set_ylabel('Altitude')
axes[1].plot(flight['Datetime'],flight['Speed'])
axes[1].set_ylabel('Speed')
axes[2].plot(flight['Datetime'],flight['Direction'])
axes[2].set_ylabel('Direction')
axes[2].set_xlabel('UTC')
fig.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(4,1,sharex='all')
axes[0].plot(flight['Datetime'],flight['Altitude'])
axes[0].set_ylabel('Alt')
axes[1].plot(flight['Datetime'],flight['Speed'])
axes[1].set_ylabel('Speed')
axes[2].plot(flight['Datetime'],flight['ke'])
axes[2].set_ylabel('KE')
axes[3].plot(flight['Datetime'],flight['pe'])
axes[3].set_ylabel('PE')
axes[3].set_xlabel('UTC')
fig.tight_layout()
plt.show()

In [ ]:
coords = [[float(b) for b in a.split(',')] for a in flight['Position']]
#This is a nest list comphrehension to convert the paired coordinates from their 
# comma seperated string format to a pair of floats

In [ ]:
flight['latitude'] = [a for a,b in coords]

In [ ]:
flight['longitude'] = [b for a,b in coords]

In [ ]:
%pip install -q folium

In [ ]:
import folium

m = folium.Map()
folium.PolyLine(zip(flight['latitude'],flight['longitude']), color="red", weight=2.5, opacity=1).add_to(m)
folium.LayerControl().add_to(m)
m.fit_bounds(m.get_bounds())
m